In [156]:
# Import Dependencies
import pandas as pd
import os
import numpy as np

# Path to resource files
path = "resources"

# Genarate a student df
students_df = pd.read_csv(os.path.join(path, 'students_complete.csv'))

# Generate a school df changed the column name 'name' to 'school'
school_df = pd.read_csv(os.path.join(path, 'schools_complete.csv')).rename(columns={"name": "school"})

# Merge two dfs into one df
stu_sch_df = pd.merge(students_df, school_df, how = "outer")
stu_sch_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [157]:
dist_sch_df = school_df[school_df["type"] == "District"]

dist_stu_sch_df = stu_sch_df[stu_sch_df["type"] == "District"]
distByType = dist_df.groupby("type")

totalStu = len(dist_stu_sch_df.index)
math_passed = len(dist_stu_sch_df[dist_stu_sch_df["math_score"] >= 60].index)
read_passed = len(dist_stu_sch_df[dist_stu_sch_df["reading_score"] >= 60].index)

dist_summary = pd.DataFrame({
    "Total Schools":[len(dist_sch_df.index)],
    "Total Students":[totalStu],
    "Total Budget":[distByType.sum().loc["District","budget"]],
    "Average Math Score":[distByType.mean().loc["District","math_score"]],
    "Average Reading Score":[distByType.mean().loc["District","reading_score"]],
    "% Passing Math":[math_passed/totalStu],
    "% Passing Reading":[read_passed/totalStu],
    "% Overall Passing Rate":[(math_passed + read_passed)/(totalStu*2)]
})

dist_summary[[
    "Total Schools",
    "Total Students",
    "Total Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
]].style.format({
    "Total Budget":"${:,d}",
    "Average Math Score":"{:.2f}",
    "Average Reading Score":"{:.2f}",
    "% Passing Math":"{:.2%}",
    "% Passing Reading":"{:.2%}",
    "% Overall Passing Rate":"{:.2%}"
})


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,7,26976,"$70,439,053,973",76.99,80.96,89.03%,100.00%,94.52%


## School Summary

In [187]:
groupBySch = stu_sch_df.groupby(["school", "type"])
# groupBySch.mean()#Total School Budget, Per Student Budget, Average Math Score, Average Reading Score

In [186]:
# groupBySch.count()#Total Student

In [185]:
# passedStu_math_bySch = stu_sch_df[stu_sch_df["math_score"] >= 60].groupby(["school", "type"])
# passedStu_math_bySch.count() # % Passing Math

In [184]:
# passedStu_read_bySch = stu_sch_df[stu_sch_df["reading_score"] >= 60].groupby(["school", "type"])
# passedStu_read_bySch.count() # % Passing Reading

In [203]:
passedStu_mathbySch = passedStu_math_bySch.count()["Student ID"]
passedStu_readbySch = passedStu_read_bySch.count()["Student ID"]

sch_summ_df = groupBySch.mean()
# totalStubySch_df
sch_summ_df["School Type"] = school_df["type"]
sch_summ_df["Total Students"] = groupBySch.count()["Student ID"]
sch_summ_df["Per Student Budget"] = sch_summ_df["budget"] / sch_summ_df["Total Students"]
sch_summ_df["Average Math Score"] = groupBySch.mean()["math_score"]
sch_summ_df["Average Reading Score"] = groupBySch.mean()["reading_score"]
sch_summ_df["% Passing Math"] = passedStu_mathbySch / sch_summ_df["Total Students"]
sch_summ_df["% Passing Reading"] = passedStu_readbySch / sch_summ_df["Total Students"]
sch_summ_df["% Overall Passing Rate"] = (passedStu_mathbySch + passedStu_readbySch)/(sch_summ_df["Total Students"]*2)
# sch_summ_df["% Overall Passing Rate"] = np.sum([sch_summ_df.loc["% Passing Math"], sch_summ_df.loc["% Passing Reading"]])

sch_summ = sch_summ_df.rename(columns={"budget": "Total School Budget"})[[
    "School Type",
    "Total Students",
    "Total School Budget",
    "Per Student Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing Rate"
    
]]
sch_summ.style.format({
    "Total School Budget":"${:,.0f}",
    "Per Student Budget":"${:,.0f}",
    "Average Math Score":"{:.2f}",
    "Average Reading Score":"{:.2f}",
    "% Passing Math":"{:.2%}",
    "% Passing Reading":"{:.2%}",
    "% Overall Passing Rate":"{:.2%}"
    
})


,,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,type,,,,,,,,,
Bailey High School,District,nan,4976,"$3,124,928",$628,77.05,81.03,89.53%,100.00%,94.76%
Cabrera High School,Charter,nan,1858,"$1,081,356",$582,83.06,83.98,100.00%,100.00%,100.00%
Figueroa High School,District,nan,2949,"$1,884,411",$639,76.71,81.16,88.44%,100.00%,94.22%
Ford High School,District,nan,2739,"$1,763,916",$644,77.10,80.75,89.30%,100.00%,94.65%
Griffin High School,Charter,nan,1468,"$917,500",$625,83.35,83.82,100.00%,100.00%,100.00%
Hernandez High School,District,nan,4635,"$3,022,020",$652,77.29,80.93,89.08%,100.00%,94.54%
Holden High School,Charter,nan,427,"$248,087",$581,83.80,83.81,100.00%,100.00%,100.00%
Huang High School,District,nan,2917,"$1,910,635",$655,76.63,81.18,88.86%,100.00%,94.43%
Johnson High School,District,nan,4761,"$3,094,650",$650,77.07,80.97,89.18%,100.00%,94.59%


## Top Performing Schools (By Passing Rate)

In [200]:
sch_summ.sort_values(by=["% Overall Passing Rate", "% Passing Math"], ascending=False).head().style.format({
    "Total School Budget":"${:,.0f}",
    "Per Student Budget":"${:,.0f}",
    "Average Math Score":"{:.2f}",
    "Average Reading Score":"{:.2f}",
    "% Passing Math":"{:.2%}",
    "% Passing Reading":"{:.2%}",
    "% Overall Passing Rate":"{:.2%}"
    
})

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,type,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,100.00%,100.00%,100.00%
Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,100.00%,100.00%,100.00%
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,100.00%,100.00%,100.00%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,100.00%,100.00%,100.00%
Shelton High School,Charter,1761,"$1,056,600",$600,83.36,83.73,100.00%,100.00%,100.00%


## Bottom Performing Schools (By Passing Rate)

In [201]:
sch_summ.sort_values(by=["% Overall Passing Rate", "% Passing Math"]).head().style.format({
    "Total School Budget":"${:,.0f}",
    "Per Student Budget":"${:,.0f}",
    "Average Math Score":"{:.2f}",
    "Average Reading Score":"{:.2f}",
    "% Passing Math":"{:.2%}",
    "% Passing Reading":"{:.2%}",
    "% Overall Passing Rate":"{:.2%}"
    
})

,,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school,type,,,,,,,,
Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,88.44%,100.00%,94.22%
Rodriguez High School,District,3999,"$2,547,363",$637,76.84,80.74,88.55%,100.00%,94.27%
Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,88.86%,100.00%,94.43%
Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,89.08%,100.00%,94.54%
Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,89.18%,100.00%,94.59%


## Math Scores by Grade

In [210]:

ndf = pd.DataFrame({
    "9th":stu_sch_df[stu_sch_df["grade"] == "9th"].groupby("school").mean()["math_score"],
    "10th":stu_sch_df[stu_sch_df["grade"] == "10th"].groupby("school").mean()["math_score"],
    "11th":stu_sch_df[stu_sch_df["grade"] == "11th"].groupby("school").mean()["math_score"],
    "12th":stu_sch_df[stu_sch_df["grade"] == "12th"].groupby("school").mean()["math_score"]
})
ndf
# .groupeby("school").mean()
# groupBySch_ = stu_sch_df.groupby(["school"])

,10th,11th,12th,9th
school,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


In [212]:
ndf_reading = pd.DataFrame({
    "9th":stu_sch_df[stu_sch_df["grade"] == "9th"].groupby("school").mean()["reading_score"],
    "10th":stu_sch_df[stu_sch_df["grade"] == "10th"].groupby("school").mean()["reading_score"],
    "11th":stu_sch_df[stu_sch_df["grade"] == "11th"].groupby("school").mean()["reading_score"],
    "12th":stu_sch_df[stu_sch_df["grade"] == "12th"].groupby("school").mean()["reading_score"]
})
ndf_reading


,10th,11th,12th,9th
school,,,,
Bailey High School,80.907183,80.945643,80.912451,81.303155
Cabrera High School,84.253219,83.788382,84.287958,83.676136
Figueroa High School,81.408912,80.640339,81.384863,81.198598
Ford High School,81.262712,80.403642,80.662338,80.632653
Griffin High School,83.706897,84.288089,84.013699,83.369193
Hernandez High School,80.660147,81.396140,80.857143,80.866860
Holden High School,83.324561,83.815534,84.698795,83.677165
Huang High School,81.512386,81.417476,80.305983,81.290284
Johnson High School,80.773431,80.616027,81.227564,81.260714
